# Notes on "OpenAI GPT for Python Devs"

by Aymen El Amri

1 token is about 4 letters for standard English words. 100 tokens is about 75 words. 8K tokens is about 12 pages of text. 32K tokens is about 50 pages.

In [ ]:
import os
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [ ]:
# get available models

models = [(m.id, m.created) for m in client.models.list()]
for model in sorted(models, key=lambda x: x[1]):  # sort by datetime
    print(f"{model[0]}, created: {datetime.fromtimestamp(model[1])}")



In [ ]:
model = "gpt-3.5-turbo"

messages = [
    {
    "role": "system",
    "content": "You are a smart and creative assistant."
    },
    {"role": "user",
     "content": "Who is Michael Jackson"}
]

response = client.chat.completions.create(model=model, messages=messages, max_tokens=50)
print(response.usage)
print("---"*10)

print("Short answer:", response.choices[0].message.content)
print("---"*10)

response = client.chat.completions.create(model=model, messages=messages, max_tokens=500)
print("Long answer: ", response.choices[0].message.content)
print("---"*10)

response = client.chat.completions.create(model=model, messages=messages, max_tokens=100, stop=["."])
print("Stop answer: ", response.choices[0].message.content + ".")


In [ ]:
def single_query(messages, model="chatgpt-4o-latest"):
    response = client.chat.completions.create(model=model, messages=messages)
    print(response.choices[0].message.content)


messages = [
    {"role": "system", "content": "You are an expert technical (XP) coach"},
    {"role": "user", "content": "List all XP values and practices"},
]
single_query(messages)

In [ ]:
def multiple_query(messages, n=2):
    response = client.chat.completions.create(
        model=model, messages=messages, n=n, stop=["\n"]
    )
    for choice in response.choices:
        print(f"Choice:{choice.index}")
        print(choice.message.content)
        print()


messages = [
    {"role": "system", "content": "you are an expert XP developer"},
    {"role": "user", "content": "Describe the meaning of Collective Code Ownership"},
]

multiple_query(messages, n=3)

In [ ]:
# Chaining: use gpt to build a prompt

model="chatgpt-4o-latest"

messages = [
    {"role": "system", "content": "You are an expert in japanese poetry."},
    {"role": "user", "content": "Write a concise paragraph about japanese haiku."},
]

# ask for a creative response
response = client.chat.completions.create(model=model, messages=messages, max_tokens=200, temperature=0.5, 
                                          stop=["assisstant:", "user:"])

paragraph = response.choices[0].message.content

messages = [
    {"role": "system", "content": "You are a poet."},
    {"role": "user", "content": f"Context: {paragraph}.\n Task: Create 3 haikus about water and mountains in autumn."}
]

response = client.chat.completions.create(model=model, messages=messages, max_tokens=200, temperature=1,
                                          stop=["assistant:", "user:"])

haikus = response.choices[0].message.content

print(f"Context:\n{paragraph}\n")
print(f"Result:\n{haikus}\n")


In [ ]:
# streaming

model = "chatgpt-4o-latest"

messages = [
    {"role": "system", "content": "You are a fantasy writer."},
    {"role": "user", "content": "Once upon a time "}
]

response = client.chat.completions.create(model=model, messages=messages, max_tokens=200, temperature=0.5, stream=True)

for message in response:
    content = message.choices[0].delta.content
    if content:
        print(content, end="")
